In [1]:
import os
os.getcwd()
import sys 
sys.path.append('/data/users/miroojin/saksham/watermark-adapters')
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

In [2]:
import json
from src.mbmark import MbMark, Mode
from src.gaussmark import GaussMark
from sklearn.metrics import roc_auc_score
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datasets import load_dataset, load_from_disk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.stats
torch.manual_seed(42)
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.interpolate import interp1d


In [3]:
model_name = "mistralai/Mistral-7B-v0.3"
model_name = "meta-llama/Llama-2-7b-hf"
model_suffix = model_name.split("/")[-1]


BATCH_SIZE = 64
print(f"Batch size: {BATCH_SIZE}")
print(f"Model name: {model_name}")


Batch size: 64
Model name: meta-llama/Llama-2-7b-hf


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    device_map="auto")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype=torch.bfloat16)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:


watermark_type = "mb"
seed = 15485863
# seed = 12997009
# seed = 22983996
if watermark_type == "gaussmark":
    watermark = GaussMark(sigma=0.04,
                          tokenizer=tokenizer, model=model, seed=seed, target_param_name="model.layers.27.mlp.up_proj.weight")
elif watermark_type == "mb":
    k = 235
    final_matrix_path = f"../saved_models/openwebtext_{model_suffix}/selector_matrix_k{k}.pth"
    final_weight = torch.load(final_matrix_path)
    watermark = MbMark.mb(
        delta=1.0,
        gamma=0.25,
        seed=seed,
        final_weight=final_weight,
        model=model,
        tokenizer=tokenizer,
        unembedding_param_name="lm_head",
        mode=Mode.Generate,
    )

In [6]:
watermarked_model = watermark.model

if watermark_type == "mb":
    watermarked_model_name = f"{model_name}-watermarked-greenlist-bias-k{k}-seed{seed}"
if watermark_type == "gaussmark":
    watermarked_model_name = f"{model_name}-watermarked-gaussmark-sigma{watermark.sigma}-seed{seed}"

print(f"Watermarked model name: {watermarked_model_name}")

# Save the model to /pool.ssd/assets/models/
watermarked_model_name = os.path.join(
    "/pool.ssd/assets/models/", watermarked_model_name)
if not os.path.exists(watermarked_model_name):
    os.makedirs(watermarked_model_name)
# Save the model to the specified directory
watermarked_model.save_pretrained(watermarked_model_name)
tokenizer.save_pretrained(watermarked_model_name)
print(f"Watermarked model saved to {watermarked_model_name}")

Watermarked model name: meta-llama/Llama-2-7b-hf-watermarked-greenlist-bias-k235-seed15485863
[2025-07-29 14:37:28,574] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/miroojin/miniconda3/envs/venv/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/miroojin/miniconda3/envs/venv/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Watermarked model saved to /pool.ssd/assets/models/meta-llama/Llama-2-7b-hf-watermarked-greenlist-bias-k235-seed15485863
